# Nurgle

> "Buboes, phlegm, blood and guts! Boils, bogeys, rot and pus! Blisters, fevers, weeping sores! From your wounds the fester pours."
> 
> -- <cite>The Chant of Nurgle</cite>

The following analyses require `jupyterthemes`, `numpy`, `pandas`, and `plotly`. Install them with `pip`.

In [ ]:
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('grade3')

In [ ]:
import os
PREFIX = os.environ.get('PWD', '.')
# PREFIX = "../build/outputs"

In [ ]:
import numpy
import pandas

In [ ]:
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

## Inputs

Reading input files for the simulation.

In [ ]:
compounds = pandas.read_csv(os.path.join(PREFIX, "compounds.csv"))
num_compounds = compounds.shape[0]
print('[{}] compounds were loaded.'.format(num_compounds))
metabolism = pandas.read_csv(os.path.join(PREFIX, "metabolism.csv"))
print('[{}] reactions were loaded.'.format(metabolism.shape[0]))

## Metabolism

Here shows profiles and analyses related to the metabolic pathway and chemical compounds.

### Concentration of Compounds

Reading timecourse data first.

In [ ]:
timecourse = pandas.read_csv(os.path.join(PREFIX, "timecourse.csv"))
timecourse = timecourse.rename(columns={timecourse.columns[0]: "Time"})
concentrations = pandas.DataFrame(timecourse.values[: , : num_compounds+2], timecourse.index, timecourse.columns[ : num_compounds+2])
indices = [0] + list(range(num_compounds+2, timecourse.shape[1]))
fluxes = pandas.DataFrame(timecourse.values[: , indices], timecourse.index, timecourse.columns[indices])

Plotting concentrations of compounds.

In [ ]:
def plot1(df, filename, indices=None, nsteps=10, rescaled=False, xlabel="", ylabel=""):
    if indices is None:
        (m, _) = df.shape
        indices = range(0, m, m // nsteps)
    if rescaled:
        func = lambda idx: df.iloc[idx, 1: ] / df.iloc[0, 1: ]
    else:
        func = lambda idx: df.iloc[idx, 1: ]

    ymin, ymax = +numpy.inf, -numpy.inf
    for idx in indices:
        y = df.iloc[idx, 1: ] / df.iloc[0, 1: ]
        ymin, ymax = min(ymin, min(func(idx))), max(ymax, max(func(idx)))

    (ymin, ymax) = ymin - (ymax - ymin) / 15, ymax + (ymax - ymin) / 15

    scatters = [
        dict(
            y=func(idx),
            text=df.columns[1: ],
            mode='markers',
            marker=dict(
                size='12', color=func(idx), colorscale='Viridis', showscale=True,
                cmin=ymin, cmax=ymax,
                line=dict(width=1)
                ),
            visible=False
            )
        for idx in indices]
    scatters[0]['visible'] = True

    steps = []
    for i, idx in enumerate(indices):
        step = dict(
            method='restyle',
            label='{}'.format(df.iloc[idx, 0]),
            args=['visible', [False] * len(scatters)],
            )
        step['args'][1][i] = True  # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [
        dict(
            active=0,
            currentvalue=dict(prefix="Time="),
            pad=dict(t=50),
            steps=steps
            )
        ]

    layout = dict(
        hovermode= 'closest',
        xaxis= dict(title=xlabel),
        yaxis=dict(title=ylabel, range=(ymin, ymax)),
        showlegend= False,
        sliders=sliders,
        height=600
        )

    fig = dict(data=scatters, layout=layout)
    iplot(fig, filename=filename)

In [ ]:
plot1(concentrations, "concentration_markers", nsteps=15, xlabel="Compound", ylabel="Concentration")

Plotting time series of compound concentrations.

In [ ]:
def plot2(df, filename, ngroups=20, lenlabel=30, rescaled=False, xlabel="", ylabel=""):
    indices = list(range(1, df.shape[1]))
    tick = len(indices) // (ngroups - 1)
    if rescaled:
        func = lambda idx: df.iloc[: , idx] / df.iloc[0, idx]
    else:
        func = lambda idx: df.iloc[: , idx]

    ymin, ymax = +numpy.inf, -numpy.inf
    for idx in indices:
        ymin, ymax = min(ymin, min(func(idx))), max(ymax, max(func(idx)))

    (ymin, ymax) = ymin - (ymax - ymin) / 15, ymax + (ymax - ymin) / 15

    scatters = [
        dict(
            x=df.iloc[: , 0],
            y=func(idx),
            mode='lines',
            name=df.columns[idx][: lenlabel],
            visible=(idx < tick)
            )
        for idx in indices]
    steps = []
    for i in range(ngroups):
        step = dict(
            method='restyle',
            label=i + 1,
            args=['visible', [(i * tick <= j < (i + 1) * tick) for j in range(len(scatters))]],
            )
        if any(step['args'][1]):
            steps.append(step)

    sliders = [
        dict(
            active=0,
            pad=dict(t=50),
            steps=steps,
            currentvalue=dict(prefix='Group')
            )
        ]

    layout = dict(
        hovermode= 'closest',
        xaxis= dict(title=xlabel),
        yaxis=dict(title=ylabel, range=(ymin, ymax)),
        # showlegend= False,
        sliders=sliders,
        height=600
        )

    fig = dict(data=scatters, layout=layout)
    iplot(fig, filename=filename)

In [ ]:
plot2(concentrations, "concentration_lines", xlabel="Time", ylabel="Concentration")

In [ ]:
plot1(fluxes, "flux_markers", nsteps=15, rescaled=True, xlabel="Reaction", ylabel="Relative Flux")

In [ ]:
plot2(fluxes, "flux_lines", rescaled=True, xlabel="Time", ylabel="Relative Flux")